In [2]:
import numpy as np
import pandas as pd

import glob

import datetime

import re

import os

# Chase statements

## functions

In [201]:
def add_20 (string):
    """
    makes 2 digit 2000's years ('17 or '10) into the 4 digit equivalent. If it's already 4 digit, leave it alone.
    
    Takes (string) which is the full date string of format mm/dd/yy or mm/dd/yyyy
    """
    #check if it's a 2 digit year
    if string[-3] == '/':
        
        return string[:-2] + '20' + string[-2:]
    
    #if it's already a 4 digit year, return as is
    elif string[-3] == '0':
        
        return string

In [30]:
def get_dtime (string):
    """
    converts strings to datetime objects using datetime.strptime mostly just wrote this so I could .apply() it in pandas
    method chains
    
    takes (string) which is the full date of format mm/dd/yyyy (can't be 2 digit year)
    """
    return datetime.datetime.strptime(string, '%m/%d/%Y')

In [299]:
def open_close (df_from_pdf):
    """
    produces a list of series slices [opening datetime, closing datetime] by searching for the (hopefully)
    single row where "opening/closing date" is listed, hopefully with the actual values as a separate cell
    than "opening/closing date" string
    ---Takes---
    df_from_pdf : a whole pdf derived credit card statement as a dataframe
    
    ---Returns---
    dates  :  [opening datetime, closing datetime] as series slices (I hope this is ok)
    """
    
    #copy so not mod orig
    df = df_from_pdf.copy()
    
    #get just the actual data in the row with opening and closing date
    df = df.dropna(axis=0, how='all').loc[df['Unnamed: 0'] == 'Opening/Closing Date', :].dropna(axis=1, how='all')
    
    #columns are default column titles that are annoying, rename to numbers
    df.columns = [i for i in range(len(df.columns))]
    
    #df now has 2 columns [0] = 'opening/closing date', [1] = 'mm/dd/yy - mm/dd/yy'
    open_close = df[1].str.split('-', expand=True)
    
    dates = []
    for i in range(len(open_close.columns)):
        dates.append(open_close[i].str.strip().apply(add_20).apply(get_dtime).values[0])
    
    #dates has open date first and close date second
    return dates

In [300]:
def is_date_string (entry):
    
    """
    decides whether or not a string matches the pattern 'mm/dd...'
    just checks if that patterns is present at the beginning of the string, doesnt check for end of string
    because statements sometimes have concatenated a bunch of strings into one giant entry that just begins
    with mm/dd
    """
    
    if isinstance(entry, str):
    
        bb = re.match(r"([0-9]{1,2}?)/([0-9]{1,2}?)", entry)

        if bb:
            return True
        else:
            return False
    else:
        return False

In [301]:
def date_desc_price (row_of_df):
    
    """
    takes a row of a dataframe containing one transaction's information
    the row must have at least 3 separate columns (others holding NaNs) with date, description, price
    they must be in that order
    
    returns a dictionary with {'date':date, 'desc':description, 'price':price} for use in making dataframes later
    """
    
    #have to drop on index because a row slice is also a series object which doesn't have multiple columns
    row = row_of_df.dropna(axis='index', how='all')
    
    #create tuples that pair date/desc/price to the appropriate item extracted from the row
    #and make a dictionary where date/desc/price : appropriate data. This makes it easier to
    #make dataframes later
    date_desc_price_dict = {i:j for (i,j) in zip(['date', 'desc', 'price'], [item for item in row])}
    

    return date_desc_price_dict

In [302]:
def expand_all_one_cell (df_row):
    """
    if an entry has all the info smashed into the first ('date') cell in the excel sheet,
    this function pulls out the date, desc and price and returns a dictionary that can be added
    into the cleaned transaction dataframe
    """
    
    entry = df_row['date']
    
    sp = ' '
    spaces = [pos for (pos, char) in enumerate(entry) if char == sp]
    
    first_space = spaces[0]
    last_space = spaces[-1]
    
    date = entry[:first_space]
    
    try:
        price = float(entry[last_space:])
    except:
        price = entry[last_space:]
    
    desc = entry[first_space:last_space].strip()
    
    
    return {i:j for (i,j) in zip(['date', 'desc', 'price'], [date, desc, price])}

In [303]:
def expand_2_in_one (df_row):
    
    entry = df_row['date']
    
    sp = ' '
    spaces = [pos for (pos, char) in enumerate(entry) if char == sp]
    
    first_space = spaces[0]
    
    date = df_row['date'][:first_space]
    desc = df_row['date'][first_space:].strip()
    price = df_row['desc']
    
    return {i:j for (i,j) in zip(['date', 'desc', 'price'], [date, desc, price])}

In [304]:
def remove_n_line (df):
    """
    sometimes the first entry in the date column is concatenated with a newline ('\n') and Purchases
    This splits at the new line and takes just the date
    """
    for row in df.index:
    
        entry = df.loc[row, 'date'] 

        if '\n' in entry:
            df.loc[row, 'date'] = entry.split('\n')[0]
        else:
            pass
        
    return df

In [305]:
def extract_all_date_desc_price (raw_df):
    
    """
    takes in a raw statement df read in from the xlsx file, searches using regex for dates of form 'mm/dd' in the
    first column, then extracts just those rows, cleans them up, uses function date_desc_price() to attempt to
    extract and label each piece of information appropriately and returns a dataframe with just the transaction
    information. Because of variable formatting in the parent xlsx, there will be errors, which I clean up later.
    """
    
    #get the indices in the statement where the first columns entry is a date, which should identify only transaction rows
    indx_where_trans = raw_df['Unnamed: 0'].apply(is_date_string)

    #get these transactions as a slice with a fresh index and no junky NaNs
    just_trans = (raw_df.loc[indx_where_trans, :]
                  .dropna(axis='columns', how='all')
                  .reset_index(drop=True)
                 )

    #make a list of dictionaries that hold all the date/desc/price info
    trans_dicts=[]
    for row in just_trans.index:

        trans_dicts.append(date_desc_price(just_trans.loc[row]))
        
    #make a nice clean dataframe with the information you want
    poss_err_trans = remove_n_line(pd.DataFrame(trans_dicts))
    
    return poss_err_trans

In [306]:
def correct_squishing_errors (date_desc_price_df):
    """
    there is variability in the parent xlsx documents that concatenates description and sometimes price to the date
    column, this checks for these errors by looking for NaN's in the transactions dataframe and then doing the proper
    correction based on the NaN pattern. The result is an dataframe with all the same transactions, but with concat
    errors fixed. The index is out of order as a result of this, can easily sort to fix.
    """
    
    for index in date_desc_price_df.index:
        
        #get a single row, may have NaNs in fields that have been concatenated into fields to the left
        h = date_desc_price_df.loc[index]

        #NaN's aren't equal to each other so this test will check for them
        #without triggering the weird np.isnan error when checking strings
        test = [item==item for item in h]
        
        #complete row
        if test == [True, True, True]:
            pass
        #date and desc squashed together, price is in desc
        elif test == [True, True, False]:
            date_desc_price_df = pd.concat([date_desc_price_df, pd.DataFrame(expand_2_in_one(h), index=[index])], axis='index')
        #all are squashed in first column
        elif test == [True, False, False] or test == [True]:
            date_desc_price_df = pd.concat([date_desc_price_df, pd.DataFrame(expand_all_one_cell(h), index=[index])], axis='index')
            
    date_desc_price_df = date_desc_price_df.dropna(axis='index', how='any')
    
    return date_desc_price_df

In [307]:
def remove_YEN (trans_df):
    """
    due to japan trip, there are some lines that have a date and the word "YEN" in them, but arent transactions
    remove these
    """
    
    for row in trans_df.index:
        #get a row
        r = trans_df.loc[row]
        
        for x in r:
            if 'YEN' in str(x):
                trans_df = trans_df.drop(row, axis=0)
    
    return trans_df

In [308]:
def complete_trans_date (trans_df):
    """
    transaction dates dont have years right now, sometimes transactions are from multiple years 
    (some are in 12/dd/2016, some are in 01/dd/2017). Add the proper year to the date column and
    extract that date to datetime
    """
    
    #check if this statement spans the change of a year
    c = trans_df.loc[0, 'st close'].year
    o = trans_df.loc[0, 'st open'].year
    
    year_diff = c - o
    
    #the statement stays within the same year
    if year_diff == 0:
        #doesn't matter which year to append
        trans_df['date'] = trans_df['date'] + '/' + str(c)
        
    #we change over years in this statement
    elif year_diff == 1:
        #where are the january transactions that should get the updated date
        where = ['01/' in d for d in trans_df['date']]
        
        #replace the january transactions with the closing date's year, which should be advanced
        trans_df.loc[where, 'date'] = trans_df.loc[where, 'date'] + '/' + str(c)
        
        #the one's that shouldn't get the advanced year
        not_where = [not x for x in where]
        
        #add the not advanced year to these dates
        trans_df.loc[not_where, 'date'] = trans_df.loc[not_where, 'date'] + '/' + str(o)
        
    else:
        raise ValueError('this statement appears to span multiple years, that doesnt make sense')
        
    trans_df['date'] = trans_df['date'].apply(get_dtime)
    
    return trans_df

In [309]:
def munge_one_statement (filepath):
    #open data
    df = pd.read_excel(filepath)

    #get the opening and closing dates as a list [open, close]
    o_c = open_close(df)
    
    #get the cleaned up, but still error prone transctions df
    poss_err_trans = extract_all_date_desc_price(df)
    
    #correct the concatenation errors
    trans_clean = correct_squishing_errors(poss_err_trans)
    
    #add the statement association information
    trans_clean['st open'] = o_c[0]
    trans_clean['st close'] = o_c[1]
    
    #remove any lines that have "YEN" because they aren't transactions
    trans_clean = remove_YEN(trans_clean)
    
    #complete and make datetime of the transaction date column
    trans_clean = complete_trans_date(trans_clean)
    
    return trans_clean

In [310]:
def all_sts_together (directory_with_st_xlsx):
    
    sts = glob.glob(directory_with_st_xlsx + '/*x8723-.xlsx')
    
    all_sts = []
    for statement_xlsx in sts:
        
        all_sts.append(munge_one_statement(statement_xlsx))
        
    full_hist = pd.concat(all_sts, axis=0)
    
    return full_hist

### work

In [333]:
dir_path = '../../../Finances/cc info/chase(-8723) statements/csv'

In [334]:
hist = all_sts_together(dir_path)

C:\Users\geeze\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [335]:
#last statement was all buggy, so i just manually got the info out
last = complete_trans_date(pd.read_excel('../../../Finances/cc info/chase(-8723) statements/csv/clean st 20181008.xlsx'))

In [336]:
#put together
full_hist = pd.concat([hist, last], axis=0)

In [337]:
full_hist = full_hist.sort_values(by='date').reset_index(drop=True)
full_hist['card'] = 'chase -8723'

In [342]:
full_hist.loc[full_hist['desc'] == 'AUTOMATIC PAYMENT - THANK YOU', :]

,date,desc,price,st open,st close,card
28,2016-08-05,AUTOMATIC PAYMENT - THANK YOU,-285.91,2016-07-09,2016-08-08,chase -8723
46,2016-09-05,AUTOMATIC PAYMENT - THANK YOU,-1401.21,2016-08-09,2016-09-08,chase -8723
70,2016-10-05,AUTOMATIC PAYMENT - THANK YOU,-333.94,2016-09-09,2016-10-08,chase -8723
88,2016-11-04,AUTOMATIC PAYMENT - THANK YOU,-999.56,2016-10-09,2016-11-08,chase -8723
107,2016-12-05,AUTOMATIC PAYMENT - THANK YOU,-453.86,2016-11-09,2016-12-08,chase -8723
141,2017-01-05,AUTOMATIC PAYMENT - THANK YOU,-395.25,2016-12-09,2017-01-08,chase -8723
164,2017-02-05,AUTOMATIC PAYMENT - THANK YOU,-695.5,2017-01-09,2017-02-08,chase -8723
183,2017-03-05,AUTOMATIC PAYMENT - THANK YOU,-833.92,2017-02-09,2017-03-08,chase -8723
210,2017-04-05,AUTOMATIC PAYMENT - THANK YOU,-967.29,2017-03-09,2017-04-08,chase -8723
224,2017-05-05,AUTOMATIC PAYMENT - THANK YOU,-1126.64,2017-04-09,2017-05-08,chase -8723


In [340]:
full_hist.to_csv('../../../Finances/cc info/chase(-8723) statements/chase_up_to_20181008.csv', index=False)

# Citi statements

In [137]:
def strip_comma (entry):
    #if the entry is NOT a null (nan)
    if not pd.isnull(entry):

        if ',' in entry:
            
            entry = entry.replace(',', '')

    return entry

In [148]:
def make_price (df):
     
    #work with the credit info
    cre = df['Credit']
    
    #where to flip credit to negative debit
    i = cre.dropna().index
    
    df.loc[i, 'Debit'] = '-' + df.loc[i, 'Credit'].apply(strip_comma)
    
    #rename for consistent column names
    df = df.rename(columns = {'Debit' : 'price'})
    
    #get rid of commas
    df['price'] = df['price'].apply(strip_comma).astype(float)

    df = df.drop('Credit', axis='columns')
    
    
    return df

In [144]:
def fix_date_desc (df):
    
    #using get_dtime from chase munge
    df['date'] = df['date'].apply(get_dtime)
    
    df['desc'] = df['desc'].str.split('\n', expand=True)[0]
    
    df.loc[df['desc'].str.contains('AUTOPAY'), 'desc'] = 'AUTOMATIC PAYMENT - THANK YOU'
    
    return df

In [152]:
def munge_citi_hist (file):
    
    df = pd.read_csv(file)
    
    df = make_price(df)
    
    df = df.rename(columns = {'Description' : 'desc', 'Date' : 'date'}).drop('Status', axis = 'columns')
    
    df = fix_date_desc(df)
    
    df['card'] = 'citi -6845'
    
    return df

In [155]:
d = '../../../Finances/cc info/citi(-6845) history/*.csv'
files = glob.glob(d)

In [156]:
full_hist = []

for file in files:
    
    df_clean = munge_citi_hist(file)
    
    full_hist.append(df_clean)
    
citi_hist = pd.concat(full_hist, axis='index')

In [160]:
citi_hist.to_csv('../../../Finances/cc info/citi(-6845) history/citi_up_to_20181026.csv', index=False)

# Discover

In [189]:
def standard_cols (df):
    
    return (df.rename(columns = {'Trans. Date' : 'date', 'Description' : 'desc', 'Amount' : 'price', 'Category' : 'cat'})
            .drop('Post Date', axis='columns')
           )

In [190]:
def replace_autopay (df):
    
    i = df['desc'].str.contains('DIRECTPAY')
    
    df.loc[i, 'desc'] = 'AUTOMATIC PAYMENT - THANK YOU'
    
    return df

In [191]:
def munge_disc_hist (file):
    
    df = pd.read_csv(file)
    
    df = standard_cols(df)
    
    df = replace_autopay(df)
    
    df['date'] = df['date'].apply(get_dtime)
    
    df['card'] = 'discover -1362'
    
    return df

In [192]:
d = '../../../Finances/cc info/discover(-1362) statements/*Statement*.csv'
files = glob.glob(d)

In [193]:
full_hist = []

for file in files:
    
    df_clean = munge_disc_hist(file)
    
    full_hist.append(df_clean)
    
disc_hist = pd.concat(full_hist, axis='index')

In [194]:
disc_hist.to_csv('../../../Finances/cc info/discover(-1362) statements/disc_up_to_20181002.csv', index=False)

# Amex

In [238]:
def make_approx_dates (df):
    
    for i in df.index:
    
        year = df.loc[i, 'st close'].year
        mo = df.loc[i, 'st close'].month
        #just make all transactions occur on the first of the month
        #since majority of each statement period is in that month
        day = '01'

        df.loc[i, 'date'] = str(mo) + '/' + day + '/' + str(year)
        
    df['date'] = df['date'].apply(get_dtime)
        
    return df

In [239]:
h = pd.read_excel('../../../Finances/cc info/amex(-2002) history.xlsx')

In [240]:
h = h.rename(columns = {'period' : 'date', 'notes' : 'desc', 'amnt' : 'price'})
h['card'] = 'amex -2002'

In [241]:
h['st open'] = h['date'].str.split('-', expand=True)[0].str.replace(' ', '').apply(add_20).apply(get_dtime)

h['st close'] = h['date'].str.split('-', expand=True)[1].str.replace(' ', '').apply(add_20).apply(get_dtime)

In [242]:
h = make_approx_dates(h)

In [245]:
h.to_csv('../../../Finances/cc info/amex(-2002)_TIDY.csv', index=False)

## Concatenate all cc history

In [5]:
os.chdir(r'C:\Users\geeze\Documents\Finances\cc info')

In [79]:
citi = r'C:\Users\geeze\Documents\Finances\cc info\citi(-6845) history\citi_up_to_20181026.csv'
disc = r'C:\Users\geeze\Documents\Finances\cc info\discover(-1362) statements\disc_up_to_20181002.csv'
amex = r'C:\Users\geeze\Documents\Finances\cc info\amex(-2002)_TIDY.csv'
chase = r'C:\Users\geeze\Documents\Finances\cc info\chase(-8723) statements\chase_up_to_20181008.csv'

ccs = [citi, disc, amex, chase]

In [80]:
hists = []

for cc in ccs:
    hist = pd.read_csv(cc)
    
    hists.append(hist)
    
hists

[           date                                      desc    price        card
 0     4/21/2017  LA TRANSIT AVIATIONQPS LOS ANGELES   CA      1.75  citi -6845
 1     4/23/2017  CHEVRON 0090199        ALHAMBRA      CA     25.33  citi -6845
 2     4/27/2017  LA CITY PARKING METER  LOS ANGELES   CA      0.25  citi -6845
 3     4/27/2017  LA CITY PARKING METER  LOS ANGELES   CA      6.00  citi -6845
 4     4/27/2017  AMAZON MKTPLACE PMTS   AMZN.COM/BILL WA     31.75  citi -6845
 5     4/29/2017  REO 0570-013-456       0570013456    GBR     5.60  citi -6845
 6      5/4/2017  AMAZON MKTPLACE PMTS   AMZN.COM/BILL WA     72.97  citi -6845
 7      5/6/2017  AMAZON MKTPLACE PMTS   AMZN.COM/BILL WA     72.97  citi -6845
 8      5/8/2017  CHEVRON 0090199        ALHAMBRA      CA     29.08  citi -6845
 9     5/11/2017  BLAZING STONE PIZZA    LOS ANGELES   CA     16.09  citi -6845
 10    5/12/2017  MAGNOLIA HOUSE         PASADENA      CA    190.00  citi -6845
 11    5/17/2017  MEMBERSHIP FEE MAY 17-

In [81]:
full_hist = pd.concat(hists)

C:\Users\geeze\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [82]:
full_hist['date'] = full_hist['date'].apply(get_dtime)

In [83]:
full_hist = (full_hist.reindex(columns = ['date', 'price', 'desc', 'card', 'cat', 'st close', 'st open'])
             .sort_values(by='date')
             .reset_index(drop=True)
            )

In [86]:
last = max(full_hist['date'])

# full_hist.to_csv(r'C:\Users\geeze\Documents\Finances\cc_hist_to_{}{}{}.csv'.format(last.year, last.month, last.day), index=False)